# GR HD Equations

$\newcommand{\be}{\begin{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\grad}{{\boldsymbol{\nabla}}}$
$\newcommand{\vel}{{\boldsymbol{v}}}$
$\newcommand{\mom}{{\boldsymbol{p}}}$
$\newcommand{\ddt}[1]{{\frac{\partial #1}{\partial t}}}$
$\newcommand{\ddx}[1]{{\frac{\partial #1}{\partial x}}}$
$\newcommand{\state}{{\boldsymbol{\mathcal{U}}}}$
$\newcommand{\charge}{{\boldsymbol{U}}}$
$\newcommand{\psicharge}{{\boldsymbol{\psi}}}$
$\newcommand{\lapse}{\alpha}$
$\newcommand{\shift}{\boldsymbol{\beta}}$
$\newcommand{\rhostar}{{\rho_*}}$
$\newcommand{\tautilde}{{\tilde{\tau}}}$
$\newcommand{\Svectilde}{{\tilde{\boldsymbol{S}}}}$
$\newcommand{\rtgamma}{{\sqrt{\gamma}}}$
$\newcommand{\T}[2]{{T^{#1 #2}}}$
$\newcommand{\uvec}{{\boldsymbol{u}}}$
$\newcommand{\Vvec}{{\boldsymbol{\mathcal{V}}}}$
$\newcommand{\vfluid}{{\boldsymbol{v}_{\rm f}}}$
$\newcommand{\vVal}{{\tilde{\boldsymbol{v}}}}$

$\newcommand{\flux}{{\boldsymbol{\mathcal{F}}}}$
$\newcommand{\fluxV}{{\boldsymbol{F}}}$
$\newcommand{\source}{{\boldsymbol{\mathcal{S}}}}$
$\newcommand{\sourceV}{{\boldsymbol{S}}}$

$\newcommand{\area}{{\boldsymbol{A}}}$
$\newcommand{\normal}{{\hat{\boldsymbol{n}}}}$
$\newcommand{\pt}{{\boldsymbol{p}}}$
$\newcommand{\nb}{{\boldsymbol{n}}}$
$\newcommand{\meshv}{{\boldsymbol{w}}}$
$\newcommand{\facev}{{\boldsymbol{\tilde{w}}_{ij}}}$
$\newcommand{\facer}{{\boldsymbol{\tilde{r}}_{ij}}}$
$\newcommand{\meshr}{{\boldsymbol{r}}}$
$\newcommand{\cmr}{{\boldsymbol{c}}}$

We start out with the ** GRHD ** equations in conservative form with the state vector $\state=(\rhostar, \Svectilde, \tautilde)$:
\begin{equation}
\ddt{\state} + \grad\cdot\flux = \source,
\end{equation}
where $\rhostar = \lapse\rho\rtgamma u^0$, $\Svectilde = \rhostar h \uvec$, $\tautilde = \lapse^2\rtgamma \T00 - \rhostar$. The associated set of primitive variables are $(\rho, \vel, \epsilon)$, which are the rest mass density, fluid 3-velocity, and internal energy (measured in the rest frame).  

The flux, $\flux$ is given by
\begin{equation}
 \flux=(\rhostar \vel, \lapse\rtgamma\T{j}{\beta}g_{\beta i}, \lapse^2\rtgamma\T0j - \rhostar\vel
\end{equation}
where $\vel$ is the 3-velocity, and $\source = (0, \frac 1 2 \lapse\rtgamma \T{\lapse}{\beta}g_{\lapse\beta,i}, s)$ is the source function, and
\begin{equation}
s = \lapse\rtgamma\left[\left(\T00\beta^i\beta^j + 2\T0i\beta^j\right)K_{ij} - \left(\T00\beta^i + \T0i\right)\partial_i\lapse\right]
\end{equation}
The stress energy tensor for a perfect fluid is written as 
\begin{equation}
\T{\mu}{\nu} = \rho h u^{\mu} u^{\nu} + P g^{\mu\nu},
\end{equation}
where $h = 1 + \epsilon + P/\rho$ is the specific enthalpy and $u^{\mu}$ are the respective components of the four velocity.  

Noting that the mass $\flux$ is defined in terms of $\rhostar$ and $\vel$, we need to first find a mapping between $\vel$ and $u$.  

## Primitive to Conservative Mapping
We want to make a mapping from the primitives to conserved variables:
\begin{equation}
(\rho, \vel, \epsilon) \rightarrow (\rhostar = \lapse\rho\rtgamma u^0, \Svectilde = \rhostar h \uvec, \tautilde = \lapse^2\rtgamma \T00 - \rhostar).
\end{equation}
To do so, we first need to determine $u^0$ and $\uvec$. Noting that $g_{\mu\nu} u^{\mu} u^{\nu} = -1$, we find
\begin{equation}
u^0 = \left(-g_{00} - 2g_{i0} v^i - g_{ij}v^iv^j\right)^{-1/2},
\end{equation}
where we have used $\vel = \uvec/u^0$.  This gives me $\rhostar$ and $\uvec$.  We note that the metric is (B&S 2.122)
\begin{equation}
g_{\mu\nu} = \begin{pmatrix} 
-\lapse^2 + \shift\cdot\shift & \beta_i \\
\beta_j & \gamma_{ij}
\end{pmatrix},
\end{equation}
Lets write some code to define metric contraction on four vectors in this context:


In [1]:
import NRPy_param_funcs as par 
import indexedexp as ixp
import sympy as sp

from outputC import *
import NRPy_param_funcs as par
par.set_paramsvals_value("outputC::SIMD_enable = False")

DIM = 3
# Declare rank-2 covariant gmunu

gmunuDD = ixp.declarerank2("gmunuDD","sym01",DIM=4)

gammaDD = ixp.declarerank2("gammaDD","sym01")
components = ["xx", "xy", "xz", "yy", "yz", "zz"]
names = ""
for comp in components : 
    names = names + "mi.gam{0} ".format(comp)

gxx, gxy, gxz, gyy, gyz, gzz = sp.symbols( names)

gammaDD[0][0] = gxx
gammaDD[0][1] = gxy
gammaDD[0][2] = gxz
gammaDD[1][0] = gxy
gammaDD[1][1] = gyy
gammaDD[1][2] = gyz
gammaDD[2][0] = gxz
gammaDD[2][1] = gyz
gammaDD[2][2] = gzz

lapse, rtgamma, beta_x, beta_y, beta_z = sp.symbols( "mi.alpha mi.rtDetGamma mi.betaX mi.betaY mi.betaZ")
u10, u1x, u1y, u1z = sp.symbols("u1[0] u1[1] u1[2] u1[3]")
u20, u2x, u2y, u2z = sp.symbols("u2[0] u2[1] u2[2] u2[3]")

u1U = ixp.declarerank1("u1Vector", DIM=4)
u2U = ixp.declarerank1("u2Vector", DIM=4)

u1U[0] = u10
u1U[1] = u1x
u1U[2] = u1y
u1U[3] = u1z

u2U[0] = u20
u2U[1] = u2x
u2U[2] = u2y
u2U[3] = u2z

shiftU = ixp.declarerank1("shiftU")
shiftU[0] = beta_x 
shiftU[1] = beta_y
shiftU[2] = beta_z 

beta2 = 0
for i in range(DIM) :
    for j in range(DIM) :
        beta2 += gammaDD[i][j] * shiftU[i]*shiftU[j]
        
gmunuDD[0][0] = -lapse*lapse + beta2
for i in range(DIM) : 
    gmunuDD[i+1][0] = shiftU[i]
    gmunuDD[0][i+1] = shiftU[i]
    for j in range(DIM) : 
        gmunuDD[i+1][j+1] = gammaDD[i][j]
        
dot4Product = 0
for i in range(4):
    for j in range(4):
        dot4Product += gmunuDD[i][j]*u1U[i]*u2U[j]

str = outputC( dot4Product, "dotProduct", filename="returnstring")
print(str)

/*
 *  Original SymPy expression:
 *  "dotProduct = mi.betaX*u1[0]*u2[1] + mi.betaX*u1[1]*u2[0] + mi.betaY*u1[0]*u2[2] + mi.betaY*u1[2]*u2[0] + mi.betaZ*u1[0]*u2[3] + mi.betaZ*u1[3]*u2[0] + mi.gamxx*u1[1]*u2[1] + mi.gamxy*u1[1]*u2[2] + mi.gamxy*u1[2]*u2[1] + mi.gamxz*u1[1]*u2[3] + mi.gamxz*u1[3]*u2[1] + mi.gamyy*u1[2]*u2[2] + mi.gamyz*u1[2]*u2[3] + mi.gamyz*u1[3]*u2[2] + mi.gamzz*u1[3]*u2[3] + u1[0]*u2[0]*(-mi.alpha**2 + mi.betaX**2*mi.gamxx + 2*mi.betaX*mi.betaY*mi.gamxy + 2*mi.betaX*mi.betaZ*mi.gamxz + mi.betaY**2*mi.gamyy + 2*mi.betaY*mi.betaZ*mi.gamyz + mi.betaZ**2*mi.gamzz)"
 */
{
   const double tmp0 = 2*mi.betaX;
   dotProduct = mi.betaX*u1[0]*u2[1] + mi.betaX*u1[1]*u2[0] + mi.betaY*u1[0]*u2[2] + mi.betaY*u1[2]*u2[0] + mi.betaZ*u1[0]*u2[3] + mi.betaZ*u1[3]*u2[0] + mi.gamxx*u1[1]*u2[1] + mi.gamxy*u1[1]*u2[2] + mi.gamxy*u1[2]*u2[1] + mi.gamxz*u1[1]*u2[3] + mi.gamxz*u1[3]*u2[1] + mi.gamyy*u1[2]*u2[2] + mi.gamyz*u1[2]*u2[3] + mi.gamyz*u1[3]*u2[2] + mi.gamzz*u1[3]*u2[3] + u1[0]*u2[0]

which then gives 
\begin{equation}
u^0 = \left(\lapse^2 - \shift\cdot\shift - 2\shift\cdot\vel - \gamma_{ij}v^iv^j\right)^{-1/2},
\end{equation}

The other thing is $\uvec = u^0\vel$.  So then we can proceed and spit out the conservative variables: $\rhostar, \Svectilde$.

To get $\tau$, we note that we have defined the metric as the covariant form, e.g., lower indices.  The upper form of $g^{\mu\nu}$ is found in B&S 2.119 and is given by
\begin{equation}
g^{\mu\nu} = 
\begin{pmatrix}
-\lapse^{-2} & \lapse^{-2}\beta^i \\
\lapse^{-2}\beta^j & \gamma^{ij} - \lapse^{-2} \beta^i\beta^j
\end{pmatrix}
\end{equation}
We will need the full form in a bit, but for now we note that $\tau = \lapse^2\rtgamma T^{00} - \rhostar$, which gives
\begin{equation}
\tau = \lapse\rhostar h u^0 - P\rtgamma - \rhostar
\end{equation}

The code for this is 

In [2]:
# Declare rank-1 contravariant ("v") vector
vx, vy, vz = sp.symbols( "vx vy vz")
vU = ixp.declarerank1("vU")
vU[0] = vx
vU[1] = vy
vU[2] = vz

rho, epsilon, gamma1 = sp.symbols("rho epsilon gamma1")
        
betaDotV = 0 
for i in range(DIM) :
    for j in range(DIM) :
        betaDotV += gammaDD[i][j] * shiftU[i]*vU[j]
        
v2 = 0
for i in range(DIM) :
    for j in range(DIM) :
         v2 += gammaDD[i][j] * vU[i]*vU[j]

u0 = 1./sp.sqrt( lapse*lapse - beta2 - 2*betaDotV - v2)

uvecU = ixp.declarerank1("uvecU")
StildeU = ixp.declarerank1("StildeU")

rhostar = lapse*rtgamma*rho*u0
h = 1 + gamma1*epsilon

for i in range(DIM): 
    uvecU[i] = vU[i]*u0
    StildeU[i] = uvecU[i]*rhostar*h

P = (gamma1 - 1)*rho*epsilon
tau = lapse*rhostar*h*u0 - rtgamma*P - rhostar

str = outputC([u0, uvecU[0], uvecU[1], uvecU[2], rhostar, StildeU[0], StildeU[1], StildeU[2], tau], ["u0", "ux", "uy", "uz", "rhostar", "Sx", "Sy", "Sz", "tau"], filename="returnstring")
print(str)

/*
 *  Original SymPy expressions:
 *  "[u0 = 1.0/sqrt(mi.alpha**2 - mi.betaX**2*mi.gamxx - 2*mi.betaX*mi.betaY*mi.gamxy - 2*mi.betaX*mi.betaZ*mi.gamxz - 2*mi.betaX*mi.gamxx*vx - 2*mi.betaX*mi.gamxy*vy - 2*mi.betaX*mi.gamxz*vz - mi.betaY**2*mi.gamyy - 2*mi.betaY*mi.betaZ*mi.gamyz - 2*mi.betaY*mi.gamxy*vx - 2*mi.betaY*mi.gamyy*vy - 2*mi.betaY*mi.gamyz*vz - mi.betaZ**2*mi.gamzz - 2*mi.betaZ*mi.gamxz*vx - 2*mi.betaZ*mi.gamyz*vy - 2*mi.betaZ*mi.gamzz*vz - mi.gamxx*vx**2 - 2*mi.gamxy*vx*vy - 2*mi.gamxz*vx*vz - mi.gamyy*vy**2 - 2*mi.gamyz*vy*vz - mi.gamzz*vz**2),
 *    ux = 1.0*vx/sqrt(mi.alpha**2 - mi.betaX**2*mi.gamxx - 2*mi.betaX*mi.betaY*mi.gamxy - 2*mi.betaX*mi.betaZ*mi.gamxz - 2*mi.betaX*mi.gamxx*vx - 2*mi.betaX*mi.gamxy*vy - 2*mi.betaX*mi.gamxz*vz - mi.betaY**2*mi.gamyy - 2*mi.betaY*mi.betaZ*mi.gamyz - 2*mi.betaY*mi.gamxy*vx - 2*mi.betaY*mi.gamyy*vy - 2*mi.betaY*mi.gamyz*vz - mi.betaZ**2*mi.gamzz - 2*mi.betaZ*mi.gamxz*vx - 2*mi.betaZ*mi.gamyz*vy - 2*mi.betaZ*mi.gamzz*vz - mi.gamxx*vx*

## Conservative to Primitive Solver

We now discuss the reverse mapping from conservative to primitive variables.
Given the lapse, shift vector and $\rtgamma$, the mapping between primitive and conserved variable is straightforward.  However, the reverse is not as simple.  In GRMHD, the conservative to primitive solver is amplified by the inclusion of the magnetic field, leading to rather sophisticated root finding strategies.  The failure rates of these algorithms are low (??), but since this algorithm may be executed several times per timestep for every gridpoint, even a low failure can give unacceptable collective failure rates.  However, for purely polytropic equations of state, e.g., $P\propto\rho^{\Gamma_1}$, the convervative to primitive variable solver is greatly simplified.  

To construct the conservative-to-primitive variable solver, we restrict ourselves to polytropic equations of states
\begin{equation}
P = P_0\left(\frac{\rho}{\rho_0}\right)^{\Gamma_1} \quad\textrm{and}\quad \epsilon = \epsilon_0\left(\frac{\rho}{\rho_0}\right)^{\Gamma_1-1},
\end{equation}
where $P_0$, $\rho_0$, and $\epsilon_0$ are the fiducial pressure, density, and internal energy, and we have used the relation $P = (\Gamma_1 - 1)\rho\epsilon$.  

For such a polytropic equation of state, the energy equation is redundant and effectively we are only concerned with the continuity and momentum equations. The conservative variables of concern are $\rhostar$ and $\Svectilde$.  Noting that the shift, $\alpha$, and $\rtgamma$ are provided by the Einsteins field equation solver, we can write
\begin{equation}
u^0 = \frac{\rhostar}{\alpha\rtgamma\rho} = u^0(\rho)  \quad\textrm{and}\quad \uvec = \frac{\Svectilde}{\alpha\rtgamma\rho h} = \uvec(\rho).
\end{equation}
Noting that the four velocity $u^2 = g_{\mu\nu}u^{\mu}u^{\nu} = g^{00}u^0u^0 + 2g^{0i}u^0\uvec^i + g_{ij}\uvec^i\uvec^j = -1$, we have
\begin{equation}
 0 = f(\rho)\equiv \alpha^2\gamma\rho^2h^2 + \left(-\lapse^2 + \shift\cdot\shift\right)\rhostar^2h^2 + 2h\rhostar\shift\cdot\Svectilde + \Svectilde\cdot\Svectilde,
\end{equation}
which is an implicit equation of $\rho$, which can be inverted by standard nonlinear root finding algorithms, e.g., Newton-raphson. We will produce a function for this depends on defining $c_{s,0}^2 = \Gamma_1 P_0/\rho_0 \rightarrow c_{s,0}^2 = \Gamma_1(\Gamma_1 - 1)\epsilon_0 \rightarrow \epsilon_0 = c_{s,0}^2/(\Gamma_1(\Gamma_1 - 1))$.  

So we can find that $h = c^2 + \Gamma_1\epsilon = c^2 + c_{s,0}^2\left(\rho/\rho_0\right)^{\Gamma_1 -1}/(\Gamma_1 - 1)$. We put this all together to define a function, $f(\rho)$, whose root is zero that we will find via Newton-raphson.  


In [3]:
DIM = 3
# Declare rank-1 contravariant ("v") vector
vU = ixp.declarerank1("vU")
#shiftU = ixp.declarerank1("shiftU")
rho, gamma1, c = sp.symbols("rho gamma1 c")
Sx, Sy, Sz = sp.symbols("con[iSx] con[iSy] con[iSz]")
cs20, rho0, rhostar = sp.symbols("cs2_0 rho_0 rhostar")
# Declare rank-2 covariant gmunu
#gammaDD = ixp.declarerank2("gammaDD","sym01")
StildeU = ixp.declarerank1("StildeU")

StildeU[0] = Sx
StildeU[1] = Sy
StildeU[2] = Sz
 
gamma = rtgamma*rtgamma
lapse2 = lapse*lapse
epsilon = cs20*(rho/rho0)**(gamma1 - 1)/(gamma1 - 1)/gamma1
h = c*c + gamma1*epsilon

beta2 = 0

for i in range(DIM) :
    for j in range(DIM) :
        beta2 += gammaDD[i][j] * shiftU[i]*shiftU[j]

betaDotStilde = 0
for i in range(DIM) :
    for j in range(DIM) :
        betaDotStilde += gammaDD[i][j] * shiftU[i]*StildeU[j]
        
Stilde2 = 0 
for i in range(DIM) :
    for j in range(DIM) :
        Stilde2 += gammaDD[i][j] * StildeU[i]*StildeU[j]

f = lapse2*gamma*rho**2*h**2 + (-lapse2 + beta2)*rhostar*h**2 + 2*h*rhostar*betaDotStilde + Stilde2

outputC(f,"rootRho")


/*
 *  Original SymPy expression:
 *  "rootRho = con[iSx]**2*mi.gamxx + 2*con[iSx]*con[iSy]*mi.gamxy + 2*con[iSx]*con[iSz]*mi.gamxz + con[iSy]**2*mi.gamyy + 2*con[iSy]*con[iSz]*mi.gamyz + con[iSz]**2*mi.gamzz + mi.alpha**2*mi.rtDetGamma**2*rho**2*(c**2 + cs2_0*(rho/rho_0)**(gamma1 - 1)/(gamma1 - 1))**2 + rhostar*(c**2 + cs2_0*(rho/rho_0)**(gamma1 - 1)/(gamma1 - 1))**2*(-mi.alpha**2 + mi.betaX**2*mi.gamxx + 2*mi.betaX*mi.betaY*mi.gamxy + 2*mi.betaX*mi.betaZ*mi.gamxz + mi.betaY**2*mi.gamyy + 2*mi.betaY*mi.betaZ*mi.gamyz + mi.betaZ**2*mi.gamzz) + rhostar*(2*c**2 + 2*cs2_0*(rho/rho_0)**(gamma1 - 1)/(gamma1 - 1))*(con[iSx]*mi.betaX*mi.gamxx + con[iSx]*mi.betaY*mi.gamxy + con[iSx]*mi.betaZ*mi.gamxz + con[iSy]*mi.betaX*mi.gamxy + con[iSy]*mi.betaY*mi.gamyy + con[iSy]*mi.betaZ*mi.gamyz + con[iSz]*mi.betaX*mi.gamxz + con[iSz]*mi.betaY*mi.gamyz + con[iSz]*mi.betaZ*mi.gamzz)"
 */
{
   const double tmp0 = con[iSx]*mi.gamxy;
   const double tmp1 = 2*con[iSz];
   const double tmp2 = con[iSx]*mi.gamx

The root solve above finds $\rho$, which then allows us to get 
\begin{equation}
u^0 = \frac{\rhostar}{\alpha\rtgamma\rho}\quad\textrm{and}\quad \vel = \frac{\uvec}{u^0} = \frac{\Svectilde}{\rhostar h(\rho)}.
\end{equation}
and thus we can find the rest of the primitives.

In [4]:
#rhostar = sp.symbols("rhostar")
#StildeU = ixp.declarerank1("StildeU")
velU = ixp.declarerank1("velU")

#lapse, rtgamma, rho, gamma1, c = sp.symbols("lapse rtgamma rho gamma1 c")
cs20, rho0, rhostar = sp.symbols("cs2_0 rho_0 rhostar")

u0 = rhostar/(lapse*rtgamma*rho)
h = c*c + gamma1*epsilon

for i in range(DIM) : 
    velU[i] = StildeU[i]/(rhostar * h)
    
outputC(velU, ["vx", "vy", "vz"])

/*
 *  Original SymPy expressions:
 *  "[vx = con[iSx]/(rhostar*(c**2 + cs2_0*(rho/rho_0)**(gamma1 - 1)/(gamma1 - 1))),
 *    vy = con[iSy]/(rhostar*(c**2 + cs2_0*(rho/rho_0)**(gamma1 - 1)/(gamma1 - 1))),
 *    vz = con[iSz]/(rhostar*(c**2 + cs2_0*(rho/rho_0)**(gamma1 - 1)/(gamma1 - 1)))]"
 */
{
   const double tmp0 = gamma1 - 1;
   const double tmp1 = 1/(rhostar*(pow(c, 2) + cs2_0*pow(rho/rho_0, tmp0)/tmp0));
   vx = con[iSx]*tmp1;
   vy = con[iSy]*tmp1;
   vz = con[iSz]*tmp1;
}
